In [2]:
import pymysql
import PIL
from PIL import Image
import pandas as pd
import numpy as np
import json
import requests
import plotly.express as px

In [3]:
conn = pymysql.connect(host="127.0.0.1", user='root', password="1234", database='phonepe')
cursor = conn.cursor()

 TRANSACTION TAB

In [4]:
in_tr_yr ='2023'
in_tr_qtr='4'
#['Recharge & bill payments', 'Peer-to-peer payments','Merchant payments', 'Financial Services', 'Others']

in_tr_tr_typ='Recharge & bill payments'

cursor.execute(
                f"SELECT States, Transaction_amount FROM aggregated_transaction WHERE Years= '{in_tr_yr}' AND Quarter = '{in_tr_qtr}' AND Transaction_name = '{in_tr_tr_typ}';")
tr_tab_qry_rslt = cursor.fetchall()
df_tr_tab_qry_rslt = pd.DataFrame(np.array(tr_tab_qry_rslt), columns=['States', 'Transaction_amount'])
df_tr_tab_qry_rslt1 = df_tr_tab_qry_rslt.set_index(pd.Index(range(1, len(df_tr_tab_qry_rslt) + 1)))

In [5]:
df_tr_tab_qry_rslt1.head()

,States,Transaction_amount
1,andaman-&-nicobar-islands,185028000.0
2,andhra-pradesh,103187000000.0
3,arunachal-pradesh,515181000.0
4,assam,8011080000.0
5,bihar,39095000000.0


In [6]:

# Transaction - table query

cursor.execute(
    f"SELECT States, Transaction_count, Transaction_amount FROM aggregated_transaction WHERE Years = '{in_tr_yr}' AND Quarter = '{in_tr_qtr}' AND Transaction_name = '{in_tr_tr_typ}';")
tr_anly_tab_qry_rslt = cursor.fetchall()
df_tr_anly_tab_qry_rslt = pd.DataFrame(np.array(tr_anly_tab_qry_rslt),
                                            columns=['States', 'Transaction_count', 'Transaction_amount'])
df_tr_anly_tab_qry_rslt1 = df_tr_anly_tab_qry_rslt.set_index(pd.Index(range(1, len(df_tr_anly_tab_qry_rslt) + 1)))
df_tr_anly_tab_qry_rslt1.head()

,States,Transaction_count,Transaction_amount
1,andaman-&-nicobar-islands,327460,185028000.0
2,andhra-pradesh,100733697,103187000000.0
3,arunachal-pradesh,1491471,515181000.0
4,assam,18323202,8011080000.0
5,bihar,62582448,39095000000.0


In [7]:
# Total Transaction Amount table query

cursor.execute(
    f"SELECT SUM(Transaction_amount), AVG(Transaction_amount) FROM aggregated_transaction WHERE Years = '{in_tr_yr}' AND Quarter = '{in_tr_qtr}' AND Transaction_name = '{in_tr_tr_typ}';")
tr_am_qry_rslt = cursor.fetchall()
df_tr_am_qry_rslt = pd.DataFrame(np.array(tr_am_qry_rslt), columns=['Total', 'Average'])
df_tr_am_qry_rslt

,Total,Average
0,1.077500e+12,2.993054e+10


In [8]:
# Total Transaction Count table query
cursor.execute(
    f"SELECT SUM(Transaction_count), AVG(Transaction_count) FROM aggregated_transaction WHERE Years = '{in_tr_yr}' AND Quarter = '{in_tr_qtr}' AND Transaction_name = '{in_tr_tr_typ}';")
tr_co_qry_rslt = cursor.fetchall()   # tuple of tuple like (())
df_tr_co_qry_rslt = pd.DataFrame(np.array(tr_co_qry_rslt), columns=['Total', 'Average'])
df_tr_co_qry_rslt

,Total,Average
0,1247185223,34644033.9722


In [9]:

# GEO VISUALISATION

df_tr_tab_qry_rslt.drop(columns=['States'],axis=1 ,inplace=True)  # Drop a State column from df_tr_tab_qry_rslt

# Clone the gio data
url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
response = requests.get(url)
data1 = json.loads(response.content)     # data1 is a dict

state_names_tra = [feature['properties']['ST_NM'] for feature in data1['features']]   # Extract state names and sort them in alphabetical order
state_names_tra.sort()

df_state_names_tra = pd.DataFrame({'States': state_names_tra})  # Create a DataFrame with the state names column

df_state_names_tra['Transaction_amount'] = df_tr_tab_qry_rslt # Combine the Gio State name with df_tr_tab_qry_rslt

df_state_names_tra.to_csv('State_trans.csv', index=False)     # convert dataframe to csv file

df_tra = pd.read_csv('State_trans.csv') # Read csv
df_tra

,States,Transaction_amount
0,Andaman & Nicobar,1.850280e+08
1,Andhra Pradesh,1.031870e+11
2,Arunachal Pradesh,5.151810e+08
3,Assam,8.011080e+09
4,Bihar,3.909500e+10
5,Chandigarh,9.442490e+08
6,Chhattisgarh,1.389830e+10
7,Dadra and Nagar Haveli and Daman and Diu,9.919970e+08
8,Delhi,4.188940e+10
9,Goa,1.413110e+09


In [10]:
 # Geo plot

fig_tra = px.choropleth(
    df_tra,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM', locations='States', color='Transaction_amount',
    color_continuous_scale='thermal', title='Transaction Analysis')
fig_tra.update_geos(fitbounds="locations", visible=False)
fig_tra.update_layout(title_font=dict(size=33), title_font_color='#AD71EF', height=800)



In [11]:
#All India Transaction Analysis Bar chart #
df_tr_tab_qry_rslt1['States'] = df_tr_tab_qry_rslt1['States'].astype(str)
df_tr_tab_qry_rslt1['Transaction_amount'] = df_tr_tab_qry_rslt1['Transaction_amount'].astype(float)
df_tr_tab_qry_rslt1_fig = px.bar(df_tr_tab_qry_rslt1, x='States', y='Transaction_amount',
                                    color='Transaction_amount', color_continuous_scale='thermal',
                                    title='Transaction Analysis Chart', height=700, )
df_tr_tab_qry_rslt1_fig.update_layout(title_font=dict(size=33), title_font_color='#AD71EF')

USER TAB

In [12]:
in_us_yr='2018'
in_us_qtr='3'
# User Analysis Bar chart query
cursor.execute(f"SELECT States, SUM(Transaction_Count) FROM aggregated_user WHERE Years = '{in_us_yr}' AND Quarter = '{in_us_qtr}' GROUP BY States;")
in_us_tab_qry_rslt = cursor.fetchall()
df_in_us_tab_qry_rslt = pd.DataFrame(np.array(in_us_tab_qry_rslt), columns=['States','Transaction Count'])
df_in_us_tab_qry_rslt1 = df_in_us_tab_qry_rslt.set_index(pd.Index(range(1, len(df_in_us_tab_qry_rslt) + 1)))
df_in_us_tab_qry_rslt1.head()


,States,Transaction Count
1,andaman-&-nicobar-islands,12149
2,andhra-pradesh,5427617
3,arunachal-pradesh,45922
4,assam,784802
5,bihar,3784535


In [13]:
cursor.execute(f"SELECT SUM(Transaction_Count), AVG(Transaction_Count) FROM aggregated_user WHERE Years = '{in_us_yr}' AND Quarter = '{in_us_qtr}';")
in_us_co_qry_rslt = cursor.fetchall()
df_in_us_co_qry_rslt = pd.DataFrame(np.array(in_us_co_qry_rslt), columns=['Total', 'Average'])
df_in_us_co_qry_rslt1 = df_in_us_co_qry_rslt.set_index(['Average'])
df_in_us_co_qry_rslt1


,Total
Average,
202046.9419,80010589


In [14]:
# GEO VISUALIZATION FOR USER

# Drop a State column from df_in_us_tab_qry_rslt
df_in_us_tab_qry_rslt.drop(columns=['States'], inplace=True)
# Clone the gio data
url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
response = requests.get(url)
data2 = json.loads(response.content)
# Extract state names and sort them in alphabetical order
state_names_use = [feature['properties']['ST_NM'] for feature in data2['features']]
state_names_use.sort()
# Create a DataFrame with the state names column
df_state_names_use = pd.DataFrame({'States': state_names_use})
# Combine the Gio State name with df_in_tr_tab_qry_rslt
df_state_names_use['Transaction_count'] = df_in_us_tab_qry_rslt
# convert dataframe to csv file
df_state_names_use.to_csv('State_user.csv', index=False)
# Read csv
df_use = pd.read_csv('State_user.csv')
# Geo plot
fig_use = px.choropleth(
    df_use,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM', locations='States', color='Transaction_count',
    color_continuous_scale='thermal', title='User Analysis')
fig_use.update_geos(fitbounds="locations", visible=False)
fig_use.update_layout(title_font=dict(size=33), title_font_color='#AD71EF', height=800)

In [15]:
# ----   /   All India User Analysis Bar chart   /     -------- #
df_in_us_tab_qry_rslt1['States'] = df_in_us_tab_qry_rslt1['States'].astype(str)
df_in_us_tab_qry_rslt1['Transaction Count'] = df_in_us_tab_qry_rslt1['Transaction Count'].astype(int)
df_in_us_tab_qry_rslt1_fig = px.bar(df_in_us_tab_qry_rslt1, x='States', y='Transaction Count', color='Transaction Count',
                            color_continuous_scale='thermal', title='User Analysis Chart',
                            height=700, )
df_in_us_tab_qry_rslt1_fig.update_layout(title_font=dict(size=33), title_font_color='#AD71EF')

In [16]:
 #  bar chart-Transaction Analysis bar chart query
st_tr_st='andhra-pradesh'
st_tr_yr=2018
st_tr_qtr='2'
cursor.execute(f"SELECT Transaction_name, Transaction_amount FROM aggregated_transaction WHERE States = '{st_tr_st}' AND Years = '{st_tr_yr}' AND Quarter = '{st_tr_qtr}';")
st_tr_tab_bar_qry_rslt = cursor.fetchall()
df_st_tr_tab_bar_qry_rslt = pd.DataFrame(np.array(st_tr_tab_bar_qry_rslt),
                                            columns=['Transaction_name', 'Transaction_amount'])
df_st_tr_tab_bar_qry_rslt1 = df_st_tr_tab_bar_qry_rslt.set_index(pd.Index(range(1, len(df_st_tr_tab_bar_qry_rslt) + 1)))
df_st_tr_tab_bar_qry_rslt1

,Transaction_name,Transaction_amount
1,Recharge & bill payments,1598590000.0
2,Peer-to-peer payments,20451500000.0
3,Merchant payments,436785000.0
4,Financial Services,68952100.0
5,Others,328092000.0


In [17]:
  # table-Transaction Analysis table query

cursor.execute(f"SELECT Transaction_name, Transaction_count, Transaction_amount FROM aggregated_transaction WHERE States = '{st_tr_st}' AND Years = '{st_tr_yr}' AND Quarter = '{st_tr_qtr}';")
st_tr_anly_tab_qry_rslt = cursor.fetchall()
df_st_tr_anly_tab_qry_rslt = pd.DataFrame(np.array(st_tr_anly_tab_qry_rslt),
                                            columns=['Transaction_name', 'Transaction_count',
                                                    'Transaction_amount'])
df_st_tr_anly_tab_qry_rslt1 = df_st_tr_anly_tab_qry_rslt.set_index( pd.Index(range(1, len(df_st_tr_anly_tab_qry_rslt) + 1)))

df_st_tr_anly_tab_qry_rslt1      
            

        

,Transaction_name,Transaction_count,Transaction_amount
1,Recharge & bill payments,6456387,1598590000.0
2,Peer-to-peer payments,4745000,20451500000.0
3,Merchant payments,604847,436785000.0
4,Financial Services,225621,68952100.0
5,Others,322063,328092000.0


In [18]:
  # Total Transaction Amount table query
cursor.execute(f"SELECT SUM(Transaction_amount), AVG(Transaction_amount) FROM aggregated_transaction WHERE States = '{st_tr_st}' AND Years = '{st_tr_yr}' AND Quarter = '{st_tr_qtr}';")
st_tr_am_qry_rslt = cursor.fetchall()
df_st_tr_am_qry_rslt = pd.DataFrame(np.array(st_tr_am_qry_rslt), columns=['Total', 'Average'])
df_st_tr_am_qry_rslt

,Total,Average
0,2.288392e+10,4.576783e+09


In [19]:

# Total Transaction Count table query
cursor.execute(f"SELECT SUM(Transaction_count), AVG(Transaction_count) FROM aggregated_transaction WHERE States = '{st_tr_st}' AND Years ='{st_tr_yr}' AND Quarter = '{st_tr_qtr}';")
st_tr_co_qry_rslt = cursor.fetchall()
df_st_tr_co_qry_rslt = pd.DataFrame(np.array(st_tr_co_qry_rslt), columns=['Total', 'Average'])
df_st_tr_co_qry_rslt

,Total,Average
0,12353918,2470783.6000


In [20]:
# bar chart -State wise Transaction Analysis bar chart   /   ------ #

df_st_tr_tab_bar_qry_rslt1['Transaction_name'] = df_st_tr_tab_bar_qry_rslt1['Transaction_name'].astype(str)
df_st_tr_tab_bar_qry_rslt1['Transaction_amount'] = df_st_tr_tab_bar_qry_rslt1['Transaction_amount'].astype(
    float)
df_st_tr_tab_bar_qry_rslt1_fig = px.bar(df_st_tr_tab_bar_qry_rslt1, x='Transaction_name',
                                        y='Transaction_amount', color='Transaction_amount',
                                        color_continuous_scale='thermal',
                                        title='Transaction Analysis Chart', height=500, )
df_st_tr_tab_bar_qry_rslt1_fig.update_layout(title_font=dict(size=33), title_font_color='#AD71EF')

In [21]:
 # User Analysis Bar chart query
st_us_st='Karnataka'
st_us_yr='2019'
cursor.execute(f"SELECT Quarter, SUM(Transaction_count) FROM aggregated_user WHERE States = '{st_us_st}' AND Years = '{st_us_yr}' GROUP BY Quarter;")
st_us_tab_qry_rslt = cursor.fetchall()
df_st_us_tab_qry_rslt = pd.DataFrame(np.array(st_us_tab_qry_rslt), columns=['Quarter', 'Transaction_count'])
df_st_us_tab_qry_rslt1 = df_st_us_tab_qry_rslt.set_index(pd.Index(range(1, len(df_st_us_tab_qry_rslt) + 1)))

df_st_us_tab_qry_rslt1

,Quarter,Transaction_count
1,1,10266174
2,2,11968087
3,3,13635075
4,4,15385416


In [22]:
# Total User Count table query
cursor.execute(f"SELECT SUM(Transaction_count), AVG(Transaction_count) FROM aggregated_user WHERE States = '{st_us_st}' AND Years = '{st_us_yr}';")
st_us_co_qry_rslt = cursor.fetchall()
df_st_us_co_qry_rslt = pd.DataFrame(np.array(st_us_co_qry_rslt), columns=['Total', 'Average'])
df_st_us_co_qry_rslt

,Total,Average
0,51254752,1164880.7273


In [23]:
# -----   /   All India User Analysis Bar chart   /   ----- #
df_st_us_tab_qry_rslt1['Quarter'] = df_st_us_tab_qry_rslt1['Quarter'].astype(int)
df_st_us_tab_qry_rslt1['Transaction_count'] = df_st_us_tab_qry_rslt1['Transaction_count'].astype(int)
df_st_us_tab_qry_rslt1_fig = px.bar(df_st_us_tab_qry_rslt1, x='Quarter', y='Transaction_count', color='Transaction_count',
                                    color_continuous_scale='thermal', title='User Analysis Chart',
                                    height=500, )
df_st_us_tab_qry_rslt1_fig.update_layout(title_font=dict(size=33), title_font_color='#AD71EF')

TOP CATEGORIES

In [24]:
 # Overall top transaction
        
 #TRANSACTION TAB
top_tr_yr ='2019'  
#Top Transaction Analysis bar chart query
cursor.execute(
    f"SELECT States, SUM(Transaction_amount) As Transaction_amount FROM top_transaction WHERE Years = '{top_tr_yr}' GROUP BY States ORDER BY Transaction_amount DESC LIMIT 10;")
top_tr_tab_qry_rslt = cursor.fetchall()
df_top_tr_tab_qry_rslt = pd.DataFrame(np.array(top_tr_tab_qry_rslt),
                                        columns=['States', 'Transaction_amount'])
df_top_tr_tab_qry_rslt1 = df_top_tr_tab_qry_rslt.set_index(
    pd.Index(range(1, len(df_top_tr_tab_qry_rslt) + 1)))

 df_top_tr_tab_qry_rslt1           
            

,States,Transaction_amount
1,telangana,299804087424.0
2,karnataka,290939403904.0
3,maharashtra,245998457728.0
4,delhi,216901272512.0
5,rajasthan,154233793696.0
6,uttar-pradesh,115643912512.0
7,tamil-nadu,104638175488.0
8,madhya-pradesh,99019123840.0
9,bihar,97777828160.0
10,andhra-pradesh,85316951328.0


In [25]:
# Top Transaction Analysis table query
cursor.execute(
    f"SELECT States, SUM(Transaction_amount) as Transaction_amount, SUM(Transaction_count) as Transaction_count FROM top_transaction WHERE Years = '{top_tr_yr}' GROUP BY States ORDER BY Transaction_amount DESC LIMIT 10;")
top_tr_anly_tab_qry_rslt = cursor.fetchall()
df_top_tr_anly_tab_qry_rslt = pd.DataFrame(np.array(top_tr_anly_tab_qry_rslt),
                                            columns=['States', 'Transaction_amount',
                                                    'Transaction_count'])
df_top_tr_anly_tab_qry_rslt1 = df_top_tr_anly_tab_qry_rslt.set_index(pd.Index(range(1, len(df_top_tr_anly_tab_qry_rslt) + 1)))
df_top_tr_anly_tab_qry_rslt1

,States,Transaction_amount,Transaction_count
1,telangana,299804087424.0,179907841
2,karnataka,290939403904.0,245693445
3,maharashtra,245998457728.0,174479746
4,delhi,216901272512.0,140049306
5,rajasthan,154233793696.0,88364478
6,uttar-pradesh,115643912512.0,77894815
7,tamil-nadu,104638175488.0,53832567
8,madhya-pradesh,99019123840.0,64000331
9,bihar,97777828160.0,47720055
10,andhra-pradesh,85316951328.0,47105285


In [26]:
# All India Transaction Analysis Bar chart
df_top_tr_tab_qry_rslt1['States'] = df_top_tr_tab_qry_rslt1['States'].astype(str)
df_top_tr_tab_qry_rslt1['Transaction_amount'] = df_top_tr_tab_qry_rslt1[
    'Transaction_amount'].astype(float)
df_top_tr_tab_qry_rslt1_fig = px.bar(df_top_tr_tab_qry_rslt1, x='States', y='Transaction_amount',
                                        color='Transaction_amount', color_continuous_scale='thermal',
                                        title='Top Transaction Analysis Chart', height=600, )
df_top_tr_tab_qry_rslt1_fig.update_layout(title_font=dict(size=33), title_font_color='#AD71EF')
df_top_tr_tab_qry_rslt1_fig

USER TAB

In [27]:
#Top User Analysis bar chart query
top_us_yr='2019'
cursor.execute(f"SELECT States, SUM(RegisteredUsers) AS Top_user FROM top_user WHERE Years='{top_us_yr}' GROUP BY States ORDER BY Top_user DESC LIMIT 10;")
top_us_tab_qry_rslt = cursor.fetchall()
df_top_us_tab_qry_rslt = pd.DataFrame(np.array(top_us_tab_qry_rslt), columns=['States', 'Transaction_count'])
df_top_us_tab_qry_rslt1 = df_top_us_tab_qry_rslt.set_index(
    pd.Index(range(1, len(df_top_us_tab_qry_rslt) + 1)))
df_top_us_tab_qry_rslt1

,States,Transaction_count
1,delhi,7096401
2,karnataka,6818955
3,uttar-pradesh,6169543
4,maharashtra,6094025
5,telangana,5301668
6,haryana,5301526
7,rajasthan,4762562
8,gujarat,3814379
9,madhya-pradesh,3544116
10,bihar,3390490


In [28]:
#All India User Analysis Bar chart
df_top_us_tab_qry_rslt1['States'] = df_top_us_tab_qry_rslt1['States'].astype(str)
df_top_us_tab_qry_rslt1['Transaction_count'] = df_top_us_tab_qry_rslt1['Transaction_count'].astype(float)
df_top_us_tab_qry_rslt1_fig = px.bar(df_top_us_tab_qry_rslt1, x='States', y='Transaction_count',
                                        color='Transaction_count', color_continuous_scale='thermal',
                                        title='Top User Analysis Chart', height=600, )
df_top_us_tab_qry_rslt1_fig.update_layout(title_font=dict(size=33), title_font_color='#AD71EF')

    Query TAB

In [29]:
#Top 10 states based on year and amount of transaction"
cursor.execute(
            "SELECT DISTINCT States,Years, SUM(Transaction_amount) AS Total_Transaction_Amount FROM top_transaction GROUP BY States,Years ORDER BY Total_Transaction_Amount DESC LIMIT 10");
data = cursor.fetchall()
columns = ['States', 'Years', 'Transaction_amount']
df = pd.DataFrame(data, columns=columns, index=range(1, len(data) + 1))
df

,States,Years,Transaction_amount
1,telangana,2022,4.537919e+12
2,maharashtra,2022,2.655134e+12
3,karnataka,2022,1.985796e+12
4,madhya-pradesh,2022,1.829282e+12
5,rajasthan,2022,1.800941e+12
6,telangana,2021,1.672212e+12
7,telangana,2023,1.588499e+12
8,delhi,2022,1.561535e+12
9,karnataka,2023,1.503235e+12
10,maharashtra,2021,1.458710e+12


In [30]:
#2.Least 10 states based on year and amount of transaction":
cursor.execute(
    "SELECT DISTINCT States,Years, SUM(Transaction_amount) as Total FROM top_transaction GROUP BY States, Years ORDER BY Total LIMIT 10");
data = cursor.fetchall()
columns = ['States', 'Years', 'Transaction_amount']
df1= pd.DataFrame(data, columns=columns, index=range(1,len(data)+1))
df1

,States,Years,Transaction_amount
1,lakshadweep,2018,1.691867e+07
2,lakshadweep,2019,3.895033e+07
3,lakshadweep,2020,8.186344e+07
4,lakshadweep,2024,1.300524e+08
5,lakshadweep,2021,1.442276e+08
6,andaman-&-nicobar-islands,2018,1.719434e+08
7,lakshadweep,2022,2.132629e+08
8,ladakh,2018,2.751205e+08
9,mizoram,2018,3.018329e+08
10,lakshadweep,2023,3.976678e+08


In [31]:
#3.Top 10 States and Districts based on Registered Users":
cursor.execute("SELECT DISTINCT States, Pincodes,SUM(RegisteredUsers) AS Users FROM top_user GROUP BY States, Pincodes ORDER BY Users DESC LIMIT 10");
data = cursor.fetchall()
columns = ['States', 'Pincodes', 'RegisteredUsers']
df2 = pd.DataFrame(data, columns=columns, index=range(1, len(data) + 1))
df2

,States,Pincodes,RegisteredUsers
1,uttar-pradesh,201301,13961270
2,delhi,110059,11671444
3,karnataka,560068,10913229
4,maharashtra,421302,9876785
5,telangana,500072,9608943
6,haryana,122001,9282210
7,delhi,110092,9247767
8,delhi,110086,7999780
9,haryana,121004,7772988
10,gujarat,395006,7695123


In [32]:
#4.least 10 States and Districts based on Registered Users":
cursor.execute("SELECT DISTINCT States, Pincodes,SUM(RegisteredUsers) AS Users FROM top_user GROUP BY States, Pincodes ORDER BY Users ASC LIMIT 10");
data = cursor.fetchall()
columns = ['States', 'Pincodes', 'RegisteredUsers']
df3 = pd.DataFrame(data, columns=columns, index=range(1, len(data) + 1))
df3

,States,Pincodes,RegisteredUsers
1,lakshadweep,682558,613
2,manipur,795009,970
3,meghalaya,793004,3091
4,mizoram,796181,3297
5,goa,403801,3775
6,andaman-&-nicobar-islands,744301,4154
7,lakshadweep,682554,4296
8,manipur,795130,4957
9,jammu-&-kashmir,192301,6716
10,lakshadweep,682559,6992


In [33]:
#5.Top 10 Districts based on the Transaction Amount":
cursor.execute(
    "SELECT DISTINCT States ,District,SUM(Transaction_Amount) AS Total FROM map_transaction GROUP BY States ,District ORDER BY Total DESC LIMIT 10");
data = cursor.fetchall()
columns = ['States', 'District', 'Transaction_Amount']
df4 = pd.DataFrame(data, columns=columns, index=range(1, len(data) + 1))
df4

,States,District,Transaction_Amount
1,karnataka,bengaluru urban district,1.477074e+13
2,telangana,hyderabad district,1.018401e+13
3,maharashtra,pune district,7.448808e+12
4,rajasthan,jaipur district,6.352221e+12
5,telangana,rangareddy district,4.934054e+12
6,telangana,medchal malkajgiri district,3.949987e+12
7,andhra-pradesh,visakhapatnam district,3.227758e+12
8,andhra-pradesh,krishna district,2.639929e+12
9,bihar,patna district,2.515930e+12
10,andhra-pradesh,guntur district,2.468675e+12


In [34]:
#6.Least 10 Districts based on the Transaction Amount":
cursor.execute(
    "SELECT DISTINCT States,District,SUM(Transaction_amount) AS Total FROM map_transaction GROUP BY States, District ORDER BY Total ASC LIMIT 10");
data = cursor.fetchall()
columns = ['States', 'District', 'Transaction_amount']
df5= pd.DataFrame(data, columns=columns, index=range(1, len(data) + 1))
df5

,States,District,Transaction_amount
1,manipur,pherzawl district,1.441209e+08
2,arunachal-pradesh,dibang valley district,4.263074e+08
3,arunachal-pradesh,pakke kessang district,4.839224e+08
4,manipur,kamjong district,4.926186e+08
5,arunachal-pradesh,shi yomi district,5.393719e+08
6,jammu-&-kashmir,muzaffarabad district,5.663291e+08
7,arunachal-pradesh,kamle district,8.679320e+08
8,nagaland,longleng district,9.210598e+08
9,lakshadweep,lakshadweep district,1.022943e+09
10,meghalaya,south west khasi hills district,1.025893e+09


In [35]:
#7.Top 10 Districts based on the Transaction count":
cursor.execute(
    "SELECT DISTINCT States,District,SUM(Transaction_Count) AS Counts FROM map_transaction GROUP BY States ,District ORDER BY Counts DESC LIMIT 10");
data = cursor.fetchall()
columns = ['States', 'District', 'Transaction_Count']
df6 = pd.DataFrame(data, columns=columns, index=range(1, len(data) + 1))
df6


,States,District,Transaction_Count
1,karnataka,bengaluru urban district,12305160846
2,maharashtra,pune district,6854100535
3,telangana,hyderabad district,6303329211
4,rajasthan,jaipur district,4110791068
5,telangana,rangareddy district,3304425112
6,telangana,medchal malkajgiri district,2655079187
7,andhra-pradesh,visakhapatnam district,1872741016
8,madhya-pradesh,bhopal district,1812393105
9,maharashtra,mumbai suburban district,1801760139
10,odisha,khordha district,1789422727


In [36]:
#8.Least 10 Districts based on the Transaction count":
cursor.execute(
    "SELECT DISTINCT States ,District,SUM(Transaction_Count) AS Counts FROM map_transaction GROUP BY States ,District ORDER BY Counts ASC LIMIT 10");
data = cursor.fetchall()
columns = ['States', 'District', 'Transaction_Count']
df7= pd.DataFrame(data, columns=columns, index=range(1, len(data) + 1))
df7

,States,District,Transaction_Count
1,manipur,pherzawl district,53327
2,arunachal-pradesh,shi yomi district,100248
3,arunachal-pradesh,dibang valley district,135093
4,manipur,kamjong district,148161
5,jammu-&-kashmir,muzaffarabad district,249264
6,arunachal-pradesh,pakke kessang district,274288
7,nagaland,longleng district,324160
8,meghalaya,south west khasi hills district,328882
9,arunachal-pradesh,kurung kumey district,357157
10,mizoram,saiha district,360255


In [37]:
#9.Top Transaction types based on the Transaction Amount":
cursor.execute(
    "SELECT DISTINCT Transaction_name, SUM(Transaction_amount) AS Amount FROM aggregated_transaction GROUP BY Transaction_name ORDER BY Amount DESC LIMIT 5");
data = cursor.fetchall()
columns = ['Transaction_name', 'Transaction_amount']
df9 = pd.DataFrame(data, columns=columns, index=range(1, len(data) + 1))
df9

,Transaction_name,Transaction_amount
1,Peer-to-peer payments,1.921119e+14
2,Merchant payments,4.332506e+13
3,Recharge & bill payments,9.678982e+12
4,Others,1.483719e+11
5,Financial Services,9.190512e+10


In [38]:
#10.Top 10 Mobile Brands based on the User count of transaction":
cursor.execute(
    "SELECT DISTINCT Brands,SUM(Transaction_count) as Total FROM aggregated_user GROUP BY Brands ORDER BY Total DESC LIMIT 10");
data = cursor.fetchall()
columns = ['Brands', 'Transaction_count']
df10 = pd.DataFrame(data, columns=columns, index=range(1, len(data) + 1))
df10

,Brands,Transaction_count
1,Xiaomi,869562617
2,Samsung,671603711
3,Vivo,625415019
4,Oppo,420250245
5,Others,282950234
6,Realme,219973222
7,Apple,95947314
8,Motorola,73340734
9,OnePlus,63677211
10,Huawei,57129693
